In [8]:
!pip install backtesting
!pip install ta
!pip install yfinance


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [10]:
class SMAcross(Strategy):
    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)
    
    def next(self):
        if crossover(self.sma1,self.sma2):
            self.buy()
        elif crossover(self.sma2,self.sma1):
            self.sell()

In [11]:
df = yf.download('BTC-USD', start='2018-01-01', multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


In [12]:
df

,Close,High,Low,Open,Volume
Date,,,,,
2018-01-01,13657.200195,14112.200195,13154.700195,14112.200195,10291200000
2018-01-02,14982.099609,15444.599609,13163.599609,13625.000000,16846600192
2018-01-03,15201.000000,15572.799805,14844.500000,14978.200195,16871900160
2018-01-04,15599.200195,15739.700195,14522.200195,15270.700195,21783199744
2018-01-05,17429.500000,17705.199219,15202.799805,15477.200195,23840899072
...,...,...,...,...,...
2025-04-03,83102.828125,83909.296875,81282.101562,82487.476562,36852112080
2025-04-04,83843.804688,84696.148438,81670.750000,83100.250000,45157640207
2025-04-05,83504.796875,84207.015625,82377.734375,83844.703125,14380803631


In [13]:
bt = Backtest(df, SMAcross, cash=100000, commission=0.002, exclusive_orders=True)

C:\Users\mafia\AppData\Local\Temp\ipykernel_30936\3459263538.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, SMAcross, cash=100000, commission=0.002, exclusive_orders=True)


In [14]:
output = bt.run()

In [15]:
output

Start                     2018-01-01 00:00:00
End                       2025-04-08 00:00:00
Duration                   2654 days 00:00:00
Exposure Time [%]                    92.84099
Equity Final [$]                 149090.79229
Equity Peak [$]                  451257.92107
Commissions [$]                   15261.03965
Return [%]                           49.09079
Buy & Hold Return [%]              1069.97346
Return (Ann.) [%]                     5.64632
Volatility (Ann.) [%]                67.05381
CAGR [%]                              5.64632
Sharpe Ratio                          0.08421
Sortino Ratio                         0.13765
Calmar Ratio                          0.06794
Alpha [%]                          -180.79879
Beta                                  0.21486
Max. Drawdown [%]                   -83.10443
Avg. Drawdown [%]                   -13.22721
Max. Drawdown Duration     1456 days 00:00:00
Avg. Drawdown Duration       99 days 00:00:00
# Trades                          

In [16]:
bt.plot()

GridPlot(id='p1347', ...)

In [17]:
optim = bt.optimize(n1 = range(50,160,10),
                   n2 = range(50,160,10),
                   constraint = lambda x: x.n2 - x.n1 > 20,
                   maximize = 'Return [%]')
bt.plot()

C:\Users\mafia\Desktop\Data engineer-proj\trading bot backtest\.myenv\Lib\site-packages\backtesting\backtesting.py:1606: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
C:\Users\mafia\Desktop\Data engineer-proj\trading bot backtest\.myenv\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=2481: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\mafia\Desktop\Data engineer-proj\trading bot backtest\.myenv\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=2602: Broker canceled the relativ

GridPlot(id='p1721', ...)

In [18]:
optim

Start                     2018-01-01 00:00:00
End                       2025-04-08 00:00:00
Duration                   2654 days 00:00:00
Exposure Time [%]                    88.62095
Equity Final [$]                2350420.26646
Equity Peak [$]                  2875196.1283
Commissions [$]                   63796.81287
Return [%]                         2250.42027
Buy & Hold Return [%]               979.65515
Return (Ann.) [%]                    54.37297
Volatility (Ann.) [%]                97.27886
CAGR [%]                             54.37297
Sharpe Ratio                          0.55894
Sortino Ratio                         1.46699
Calmar Ratio                          0.68344
Alpha [%]                          2058.31363
Beta                                   0.1961
Max. Drawdown [%]                   -79.55785
Avg. Drawdown [%]                    -8.78014
Max. Drawdown Duration     1343 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                          